# **DATA PREPROCESSING BASED ON EDA INSIGHTS**

This notebook implements preprocessing steps based on the comprehensive EDA findings and recommendations. It follows the evidence-based approach from the EDA report to ensure our preprocessing aligns with the data patterns discovered.
Based on the EDA report, I will:

    ✅ Handled missing values properly.
    ✅ Handled the outliers.
    ✅ Handled the skewness.
    ✅ Created engineered features (TotalIncome, Log_TotalIncome, DTI, EMI, Flag).
    ✅ Encoded categorical variables.
    ✅ Scaled numeric data.
    ✅ Split dataset into training and testing subsets.

In [153]:
# let call all neccessary libraies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as stat 
import warnings
warnings.filterwarnings('ignore')

# Preprocessing libraries
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [154]:
# let load the data from EDA
df_process = pd.read_csv("EDA_data.csv")


## EDA_Based findings validation

In [155]:
df_process.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001003,Male,Yes,1,Graduate,No,4583.0,1508.0,128.0,360,1,Rural,N
1,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360,1,Urban,Y
2,LP001006,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.0,360,1,Urban,Y
3,LP001008,Male,No,0,Graduate,No,6000.0,0.0,141.0,360,1,Urban,Y
4,LP001011,Male,Yes,2,Graduate,Yes,5417.0,4196.0,267.0,360,1,Urban,Y


In [156]:
# let make a copy for preprocess
df= df_process.copy()

# check for missing values, it should be zero
print("")
print("--------------------------------")
print("Checking Missing Values")
print("--------------------------------")
missing_vals= df.isnull().sum()
if missing_vals.sum()>0:
    print(missing_vals[missing_vals>0])
else:
    print("No missing values as expected from EDA")

# check for duplicates
print("")
print("--------------------------------")
print("Checking duplicates rows")
print("--------------------------------")
duplicates=df.duplicated().sum()
if duplicates>0:
    print(f"The number of duplicate rows: {duplicates}")
    print(f"Percentage of duplicates:{duplicates/len(df)*100:.2f}%")

else:
    print(f"No duplicate rows")

# check skewness for the claimed variabled in EDA report as right skewed
print("")
print("--------------------------------")
print("Checking skewness of Features")
print("--------------------------------")

claimed_features=['ApplicantIncome','CoapplicantIncome','LoanAmount']
for col in claimed_features:
    skew_val= df[col].skew()
    print(f"{col} Skewness is: {skew_val:.2f}, ({"right-skewed" if skew_val > 0.5 else "approximately normal"})")




--------------------------------
Checking Missing Values
--------------------------------
No missing values as expected from EDA

--------------------------------
Checking duplicates rows
--------------------------------
No duplicate rows

--------------------------------
Checking skewness of Features
--------------------------------
ApplicantIncome Skewness is: 6.51, (right-skewed)
CoapplicantIncome Skewness is: 7.42, (right-skewed)
LoanAmount Skewness is: 2.65, (right-skewed)


## **Outlier Treatment**
IQR-capping for extreme Application Income, Coapplication Income and Loan amount

In [157]:
# let define fuction to do IQR Clipping
def IQR_Clipping(column):
    Q1= df[column].quantile(0.25)
    Q3= df[column].quantile(0.75)
    IQR= Q3-Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    # get the number of capped outliers
    outliers_count= ((df[column]<lower) | (df[column]>upper)).sum()
    if outliers_count>0:
        df[column]= np.where(df[column]<lower,lower, np.where(df[column]>upper, upper, df[column]))
    
    print(f"\nTotal outliers capped for {column}: {outliers_count}")


In [158]:
for col in ['ApplicantIncome','CoapplicantIncome','LoanAmount']:
    IQR_Clipping(col)


Total outliers capped for ApplicantIncome: 48

Total outliers capped for CoapplicantIncome: 18

Total outliers capped for LoanAmount: 39


## Handle Skewed Variables



In [159]:
# As suggested from EDA, we are using log transform bcus it is right skewed
# Log-transform skewed variables(e.g.,ApplicationIncome CoapplicationIncome and Loanamount).
df["ApplicantIncome"]= np.log1p(df["ApplicantIncome"])
df["CoapplicantIncome"]= np.log1p(df["CoapplicantIncome"])
df["LoanAmount"]= np.log1p(df["LoanAmount"])

## **Feature Engineering** - Create and interaction features

-   Total Income (TotalIncome = ApplicantIncome + CoapplicantIncome)
-   Debt-to-Income Ratio (DTI = LoanAmount / TotalIncome)
-   Equated Monthly Instalment Feature (EMI = LoanAmount / Loan_Amount_Term)

In [160]:
# total income
df["TotalIncome"]= df["ApplicantIncome"] + df["CoapplicantIncome"]

# Debt to income ratio (DTI)
df["DTI"]= df["LoanAmount"] / df["TotalIncome"]

# Equated Monthly Instalment (EMI)
df["EMI"]= df["LoanAmount"]/ df["Loan_Amount_Term"]


## **Encoded categorical variables.**

In [161]:
# let encode our categorical feature into numerical.
categorical_col= df.select_dtypes(include="object").columns
categorical_col

Index(['Loan_ID', 'Gender', 'Married', 'Education', 'Self_Employed',
       'Property_Area', 'Loan_Status'],
      dtype='object')

I will be doing label encoding for binary column and One-hot encoding for multi-category column

In [162]:
# label encoding
le= LabelEncoder()


binary_col=['Gender', 'Married', 'Education', 'Self_Employed', 'Loan_Status']
for col in binary_col:
    df[col]=le.fit_transform(df[col])

# OneHotEncoding for multi-category

df = pd.get_dummies(df,columns=["Property_Area"], drop_first=True)

# let see whats up
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,TotalIncome,DTI,EMI,Property_Area_Semiurban,Property_Area_Urban
0,LP001003,1,1,1,0,0,8.430327,7.319202,4.859812,360,1,0,15.749530,0.308569,0.013499,False,False
1,LP001005,1,1,0,0,1,8.006701,0.000000,4.204693,360,1,1,8.006701,0.525147,0.011680,False,True
2,LP001006,1,1,0,1,0,7.857094,7.765993,4.795791,360,1,1,15.623087,0.306968,0.013322,False,True
3,LP001008,1,0,0,0,0,8.699681,0.000000,4.955827,360,1,1,8.699681,0.569656,0.013766,False,True
4,LP001011,1,1,2,0,1,8.597482,8.342125,5.590987,360,1,1,16.939607,0.330054,0.015531,False,True


## **Scaled numeric data.**

In [163]:
# let normalize numeric values to help algorithms like Logistic Regression or SVM perform better.

scaler= StandardScaler()
num_cols=['ApplicantIncome','CoapplicantIncome','LoanAmount','TotalIncome','Loan_Amount_Term', 'EMI','DTI']

df[num_cols]= scaler.fit_transform(df[num_cols])
df[num_cols].std()

ApplicantIncome      1.000866
CoapplicantIncome    1.000866
LoanAmount           1.000866
TotalIncome          1.000866
Loan_Amount_Term     1.000866
EMI                  1.000866
DTI                  1.000866
dtype: float64

In [164]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,TotalIncome,DTI,EMI,Property_Area_Semiurban,Property_Area_Urban
0,LP001003,1,1,1,0,0,0.239537,0.788442,0.028269,0.273068,1,0,0.860691,-0.871572,-0.143847,False,False
1,LP001005,1,1,0,0,1,-0.534568,-1.113273,-1.411882,0.273068,1,1,-1.244501,0.812283,-0.243029,False,True
2,LP001006,1,1,0,1,0,-0.807949,0.904530,-0.112470,0.273068,1,1,0.826312,-0.884016,-0.153540,False,True
3,LP001008,1,0,0,0,0,0.731735,-1.113273,0.239338,0.273068,1,1,-1.056087,1.158335,-0.129311,False,True
4,LP001011,1,1,2,0,1,0.544983,1.054224,1.635612,0.273068,1,1,1.184260,-0.704527,-0.033152,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,LP002978,0,0,0,0,0,-0.596496,-1.113273,-1.253662,0.273068,1,1,-1.253715,0.899824,-0.232132,False,False
574,LP002979,1,1,3,0,0,0.038752,-1.113273,-2.491513,-2.479837,1,1,-1.159196,0.199409,0.244832,False,False
575,LP002983,1,1,1,0,0,1.273719,0.311817,1.517667,0.273068,1,1,0.515812,-0.297671,-0.041275,False,True
576,LP002984,1,1,2,0,0,1.159539,-1.113273,0.856215,0.273068,1,1,-0.992434,1.286483,-0.086828,False,True


In [165]:
# let see whats up
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,TotalIncome,DTI,EMI,Property_Area_Semiurban,Property_Area_Urban
0,LP001003,1,1,1,0,0,0.239537,0.788442,0.028269,0.273068,1,0,0.860691,-0.871572,-0.143847,False,False
1,LP001005,1,1,0,0,1,-0.534568,-1.113273,-1.411882,0.273068,1,1,-1.244501,0.812283,-0.243029,False,True
2,LP001006,1,1,0,1,0,-0.807949,0.904530,-0.112470,0.273068,1,1,0.826312,-0.884016,-0.153540,False,True
3,LP001008,1,0,0,0,0,0.731735,-1.113273,0.239338,0.273068,1,1,-1.056087,1.158335,-0.129311,False,True
4,LP001011,1,1,2,0,1,0.544983,1.054224,1.635612,0.273068,1,1,1.184260,-0.704527,-0.033152,False,True


## Feature Selection

In [ ]:
# As evidence from EDA, our high signals are
high_signals=['ApplicantIncome','CoapplicantIncome','LoanAmount']

# our  engineered features are:
featured_eng=['TotalIncome', 'EMI','DTI']

# so selected features will be our high signals + engineered feature
selected_features=high_signals + featured_eng

selected_features= df[selected_features]




,ApplicantIncome,CoapplicantIncome,LoanAmount,TotalIncome,EMI,DTI
0,0.239537,0.788442,0.028269,0.860691,-0.143847,-0.871572
1,-0.534568,-1.113273,-1.411882,-1.244501,-0.243029,0.812283
2,-0.807949,0.904530,-0.112470,0.826312,-0.153540,-0.884016
3,0.731735,-1.113273,0.239338,-1.056087,-0.129311,1.158335
4,0.544983,1.054224,1.635612,1.184260,-0.033152,-0.704527
...,...,...,...,...,...,...
573,-0.596496,-1.113273,-1.253662,-1.253715,-0.232132,0.899824
574,0.038752,-1.113273,-2.491513,-1.159196,0.244832,0.199409
575,1.273719,0.311817,1.517667,0.515812,-0.041275,-0.297671
576,1.159539,-1.113273,0.856215,-0.992434,-0.086828,1.286483


## **Split dataset into training and testing subsets.**

In [167]:
x= df.drop(columns=['Loan_ID','Loan_Status'])
y=df["Loan_Status"]

# let split our data
x_train,x_test,y_train,y_test= train_test_split(x,y, stratify=y, test_size=0.2, random_state=234)